# Install library

In [ ]:
!pip install pandas
!pip install openpyxl
!pip install gensim
!pip install SPARQLWrapper
!pip install xlrd==1.2.0
!pip install nltk

# Import library

In [33]:
import sys
import re
import pandas as pd
import numpy as np
from gensim.models import FastText
from gensim.models import KeyedVectors
from SPARQLWrapper import SPARQLWrapper, JSON
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword_set = set(stopwords.words('indonesian'))

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Read data

In [3]:
# sample data
path = '../example-data/'
df_prof = pd.read_excel(path + 'df_structured.xlsx')
df_prof.info()
df_prof.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 14 columns):
profile_background_color        1 non-null int64
profile_link_color              1 non-null object
profile_sidebar_border_color    1 non-null object
profile_sidebar_fill_color      1 non-null object
profile_text_color              1 non-null object
bio                             1 non-null object
name                            1 non-null object
username                        1 non-null object
followers                       1 non-null int64
friends_count                   1 non-null int64
listed_count                    1 non-null int64
favourites_count                1 non-null int64
statuses_count                  1 non-null int64
protected                       1 non-null bool
dtypes: bool(1), int64(6), object(7)
memory usage: 185.0+ bytes


,profile_background_color,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,bio,name,username,followers,friends_count,listed_count,favourites_count,statuses_count,protected
0,352726,D02B55,829D5E,99CC33,3E4415,Governor of West Java. Broadcaster of Daily Ha...,Ridwan Kamil,ridwankamil,4974848,2842,1547,22037,43924,False


In [9]:
# embedding model
path = '../embedding-model/'
ft = FastText.load_fasttext_format(path + 'fasttext.4B.id.300.epoch5.uncased.bin')
print(ft.wv[''])
ft_mei = FastText.load_fasttext_format(path + 'Fastext_model_tweet.bin')
print(ft_mei.wv[''])
w2v = KeyedVectors.load_word2vec_format(path + 'Word2Vec_400dim.txt', binary=False)
print(w2v['tes'])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 

In [77]:
# vectorizer
path = '../vectorizer/'
bio_feat_bow_vectorizer = pickle.load(open(path + 'bio_feat_bow_vec.pickle', 'rb'))
bio_feat_bow_stop_vectorizer = pickle.load(open(path + 'bio_feat_bow_stop_vec.pickle', 'rb'))

name_feat_ia_vectorizer = pickle.load(open(path + 'name_feat_ia_vec.pickle', 'rb'))
name_feat_ib_vectorizer = pickle.load(open(path + 'name_feat_ib_vec.pickle', 'rb'))
name_feat_ic_first_vectorizer = pickle.load(open('name_feat_ic_first_vec.pickle', 'rb'))
name_feat_ic_middle_vectorizer = pickle.load(open(path + 'name_feat_ic_middle_vec.pickle', 'rb'))
name_feat_ic_last_vectorizer = pickle.load(open(path + 'name_feat_ic_last_vec.pickle', 'rb'))

username_feat_bow_vectorizer = pickle.load(open(path + 'username_feat_bow_vec.pickle', 'rb'))
username_feat_char_vectorizer = pickle.load(open(path + 'username_feat_char_vec.pickle', 'rb'))

# Function

In [80]:
def get_wikidata_dict(endpoint_url, query):
    user_agent = 'WDQS-example Python/%s.%s' % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_name_tokens(df, gender):
    df_temp = df[df.gender == gender]
    arr_res = []
    for d in df_temp['nama']:
        tokens = nltk.word_tokenize(d)
        tokens = [t.lower() for t in tokens]
        arr_res = arr_res + tokens
    return list(set(arr_res))

def get_vec(vectorizer, arr_text):
    return vectorizer.transform(arr_text).toarray()

# Main

## Color

In [11]:
# extraction
color_feat = []
color_feat_abl_pbc = []
color_feat_abl_plc = []
color_feat_abl_psbc = []
color_feat_abl_psfc = []
color_feat_abl_ptc = []

color_cols = [
    'profile_background_color',
    'profile_link_color',
    'profile_sidebar_border_color',
    'profile_sidebar_fill_color',
    'profile_text_color'
]

for index, row in df_prof.iterrows():
    pbc = str(row['profile_background_color'])
    plc = str(row['profile_link_color'])
    psbc = str(row['profile_sidebar_border_color'])
    psfc = str(row['profile_sidebar_fill_color'])
    ptc = str(row['profile_text_color'])
    
    rgb1 = [int(pbc[0:2], 16) // 32, int(pbc[2:4], 16) // 32, int(pbc[4:6], 16) // 32]
    rgb2 = [int(plc[0:2], 16) // 32, int(plc[2:4], 16) // 32, int(plc[4:6], 16) // 32]
    rgb3 = [int(psbc[0:2], 16) // 32, int(psbc[2:4], 16) // 32, int(psbc[4:6], 16) // 32]
    rgb4 = [int(psfc[0:2], 16) // 32, int(psfc[2:4], 16) // 32, int(psfc[4:6], 16) // 32]
    rgb5 = [int(ptc[0:2], 16) // 32, int(ptc[2:4], 16) // 32, int(ptc[4:6], 16) // 32]
    
    color_feat.append(sum([rgb1, rgb2, rgb3, rgb4, rgb5], []))
    color_feat_abl_pbc.append(sum([rgb2, rgb3, rgb4, rgb5], []))
    color_feat_abl_plc.append(sum([rgb1, rgb3, rgb4, rgb5], []))
    color_feat_abl_psbc.append(sum([rgb1, rgb2, rgb4, rgb5], []))
    color_feat_abl_psfc.append(sum([rgb1, rgb2, rgb3, rgb5], []))
    color_feat_abl_ptc.append(sum([rgb1, rgb2, rgb3, rgb4], []))

color_feat = np.array(color_feat)
print('All color:', color_feat)
color_feat_abl_pbc = np.array(color_feat_abl_pbc)
print('Ablation background_color:', color_feat_abl_pbc)
color_feat_abl_plc = np.array(color_feat_abl_plc)
print('All link_color:', color_feat_abl_plc)
color_feat_abl_psbc = np.array(color_feat_abl_psbc)
print('All sidebar_border_color:', color_feat_abl_psbc)
color_feat_abl_psfc = np.array(color_feat_abl_psfc)
print('All sidebar_fill_color:', color_feat_abl_psfc)
color_feat_abl_ptc = np.array(color_feat_abl_ptc)
print('All text_color:', color_feat_abl_ptc)

All color: [[1 1 1 6 1 2 4 4 2 4 6 1 1 2 0]]
Ablation background_color: [[6 1 2 4 4 2 4 6 1 1 2 0]]
All link_color: [[1 1 1 4 4 2 4 6 1 1 2 0]]
All sidebar_border_color: [[1 1 1 6 1 2 4 6 1 1 2 0]]
All sidebar_fill_color: [[1 1 1 6 1 2 4 4 2 1 2 0]]
All text_color: [[1 1 1 6 1 2 4 4 2 4 6 1]]


## Bio

In [31]:
# extraction
bio_feat_bow = []
bio_feat_bow_stop = []
bio_feat_fasttext = []
bio_feat_fasttext_mei = []
bio_feat_w2v = []

for d in df_prof['bio'].fillna(''):
    bio_feat_bow.append(d)
    tokens = nltk.word_tokenize(d)
    vec_sum_ft = np.zeros(ft.vector_size)
    vec_sum_ft_mei = np.zeros(ft_mei.vector_size)
    vec_sum_w2v = np.zeros(w2v.vector_size)
    
    tokens_stop = []
    
    for t in tokens:
        t = t.lower()
        
        try:
            vec_sum_ft += ft.wv[t]
        except:
            pass
        
        try:
            vec_sum_ft_mei += ft_mei.wv[t]
        except:
            pass
        
        try:
            vec_sum_w2v += w2v.wv[t]
        except:
            pass
        
        if t not in stopword_set:
            tokens_stop.append(t)

    bio_feat_bow_stop.append(' '.join(tokens_stop))
    if len(tokens) > 0:
        vec_sum_ft = vec_sum_ft / len(tokens)
        vec_sum_ft_mei = vec_sum_ft_mei / len(tokens)
        vec_sum_w2v = vec_sum_w2v / len(tokens)
        
    bio_feat_fasttext.append(vec_sum_ft)
    bio_feat_fasttext_mei.append(vec_sum_ft_mei)
    bio_feat_w2v.append(vec_sum_w2v)
print(bio_feat_bow)
print(bio_feat_bow_stop)
print(bio_feat_fasttext)
print(bio_feat_fasttext_mei)
print(bio_feat_w2v)

['Governor of West Java. Broadcaster of Daily Happiness']
['governor of west java . broadcaster of daily happiness']
[array([ 2.29616454e-01,  2.32022300e-01, -4.72580629e-02,  3.54557215e-01,
        2.72061665e-01, -1.98238861e-01,  1.20082080e-01, -1.08769054e-01,
        9.88081050e-02,  9.92208214e-02,  1.93553222e-01, -4.42743142e-01,
        2.50536754e-01, -5.49882320e-02, -1.01235192e-01,  1.51985202e-01,
       -1.23445698e-01,  1.10331275e-01,  3.58676239e-02, -1.87953790e-01,
       -5.09244887e-03, -1.28359731e-01,  1.07102009e-01,  6.22490558e-02,
        1.23398944e-01, -3.96360084e-02, -1.37843834e-01, -1.94239910e-01,
       -4.38940442e-01,  1.97295624e-01, -2.43775972e-01, -1.88601344e-01,
       -8.34941119e-02,  1.19577813e-01,  2.71851562e-01, -2.33857926e-01,
       -1.03493498e-01, -2.69012964e-01, -1.21924798e-01, -3.14339507e-01,
        1.60772656e-01, -2.51501328e-01,  1.90861596e-01, -1.12338199e-03,
        1.65192808e-02,  1.53651417e-01, -1.45785145e-01,

In [81]:
bio_feat_bow_vec = get_vec(bio_feat_bow_vectorizer, bio_feat_bow)
bio_feat_bow_stop_vec = get_vec(bio_feat_bow_stop_vectorizer, bio_feat_bow_stop)

print(bio_feat_bow_vec)
print(bio_feat_bow_stop_vec)

[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]


## Name

Ia. Tokenize the name into bag of words

Ib. Use only first name

Ic. Parse name into first name, middle name, and last name <br>
If the name is more than 3 words, middle name has more than one value <br>
If the name is only one words, both middle and last names are null values <br>

In [43]:
# get name dictionary
endpoint_url = 'https://query.wikidata.org/sparql'

query = '''SELECT ?item ?itemLabel ?sexLabel
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P27 wd:Q252 .
  ?item wdt:P21 ?sex .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "id, en". }
}'''

results = get_wikidata_dict(endpoint_url, query)

df_wikidata = pd.DataFrame()
for index, row in pd.DataFrame(results['results']['bindings']).iterrows():
    nama = row['itemLabel']['value']
    gender = row['sexLabel']['value']

    df_wikidata = df_wikidata.append({
        'nama': nama,
        'gender': gender
    }, ignore_index=True)
    
df_wikidata = df_wikidata.drop_duplicates()
display(df_wikidata['gender'].value_counts(dropna=False))
display(df_wikidata.head(1))

tokens_pria = get_name_tokens(df_wikidata, 'laki-laki')
print(len(tokens_pria))
display(tokens_pria[0:5])

tokens_wanita = get_name_tokens(df_wikidata, 'perempuan')
print(len(tokens_wanita))
display(tokens_wanita[0:5])

21155


['sunan', 'syah', 'marwin', 'kiras', 'patinama']

13243


['syah', 'marwin', 'wira', 'gaecilia', 'kartenegara']

In [45]:
# extraction
arr_name = []
dict_name = []

arr_first_name = []
dict_first_name = []

arr_first_name_alt = []
dict_first_name_alt = []
arr_middle_name = []
dict_middle_name = []
arr_last_name = []
dict_last_name = []

for d in df_prof['name']:
    d = d.lower()
    tokens = nltk.word_tokenize(d)
    
    contains_stopword = len(set(tokens) & (stopword_set - set(['dini', 'hari']))) > 0
    if re.sub(r'[\.\s\'\_\(\)\-\|\/\@]', '', d).isalpha() and not contains_stopword:
        arr_name.append(d)
        arr_first_name.append(' '.join(tokens[0:-1]))
        if len(tokens) >= 3:
            arr_first_name_alt.append(tokens[0])        
            arr_last_name.append(tokens[-1])
            arr_middle_name.append(' '.join(tokens[1:-1]))
        elif len(tokens) == 2:
            arr_first_name_alt.append(tokens[0])
            arr_middle_name.append('')
            arr_last_name.append(tokens[-1])
        else:
            arr_first_name_alt.append(tokens[0])
            arr_middle_name.append('')
            arr_last_name.append('')

        count_pria = 0
        count_wanita = 0
        
        count_pria_first = 0
        count_wanita_first = 0

        count_pria_first_alt = 0
        count_wanita_first_alt = 0

        count_pria_middle = 0
        count_wanita_middle = 0

        count_pria_last = 0
        count_wanita_last = 0

        for t in range(len(tokens)):
            if t == 0:
                if tokens[t] in tokens_pria:
                    count_pria_first += 1
                    count_pria_first_alt += 1
                elif tokens[t] in tokens_wanita:
                    count_wanita_first += 1
                    count_wanita_first_alt += 1
                    
            elif t == len(tokens) - 1:
                if tokens[t] in tokens_pria:
                    count_pria_last += 1
                elif tokens[t] in tokens_wanita:
                    count_wanita_last += 1
                    
            else:
                if tokens[t] in tokens_pria:
                    count_pria_first += 1
                    count_pria_middle += 1
                elif tokens[t] in tokens_wanita:
                    count_wanita_first += 1
                    count_wanita_middle += 1
            
            if tokens[t] in tokens_pria:
                count_pria += 1
            elif tokens[t] in tokens_wanita:
                count_wanita += 1
        dict_name.append([count_pria, count_wanita])
        dict_first_name.append([count_pria_first, count_wanita_first])
        dict_first_name_alt.append([count_pria_first_alt, count_wanita_first_alt])
        dict_middle_name.append([count_pria_middle, count_wanita_middle])
        dict_last_name.append([count_pria_last, count_wanita_last])
        
    else:
        arr_name.append('')
        arr_first_name.append('')
        arr_first_name_alt.append('')
        arr_middle_name.append('')
        arr_last_name.append('')
        dict_name.append([0, 0])
        dict_first_name.append([0, 0])
        dict_first_name_alt.append([0, 0])
        dict_middle_name.append([0, 0])
        dict_last_name.append([0, 0])

arr_name = np.array(arr_name) 
dict_name = np.array(dict_name)

arr_first_name = np.array(arr_first_name)
dict_first_name = np.array(dict_first_name)

arr_first_name_alt = np.array(arr_first_name_alt) 
dict_first_name_alt = np.array(dict_first_name_alt)

arr_middle_name = np.array(arr_middle_name) 
dict_middle_name = np.array(dict_middle_name)

arr_last_name = np.array(arr_last_name)
dict_last_name = np.array(dict_last_name)

print(arr_name, dict_name)
print(arr_first_name, dict_first_name)
print(arr_first_name_alt, dict_first_name_alt)
print(arr_middle_name, dict_middle_name)
print(arr_last_name, dict_last_name)

['ridwan kamil'] [[2 0]]
['ridwan'] [[1 0]]
['ridwan'] [[1 0]]
[''] [[0 0]]
['kamil'] [[1 0]]


In [82]:
# vectorization

ia = get_vec(name_feat_ia_vectorizer, arr_name)
ib = get_vec(name_feat_ib_vectorizer, arr_first_name)

ic_first_name_alt = get_vec(name_feat_ic_first_vectorizer, arr_first_name_alt)
ic_middle_name = get_vec(name_feat_ic_middle_vectorizer, arr_middle_name)
ic_last_name = get_vec(name_feat_ic_last_vectorizer, arr_last_name)

ic = np.hstack((ic_first_name_alt, ic_middle_name, ic_last_name))

print(ia)
print(ib)
print(ic)

# concate with name dict feature

ia_with_dict = np.hstack((ia, dict_name))
ib_with_dict = np.hstack((ib, dict_first_name))
ic_train_with_dict = np.hstack((ic_first_name_alt, dict_first_name_alt, ic_middle_name, dict_middle_name, ic_last_name, dict_last_name))

print(ia_with_dict)
print(ib_with_dict)
print(ic_train_with_dict)

[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 2 0]]
[[0 0 0 ... 0 1 0]]
[[0 0 0 ... 0 1 0]]


## Username

In [83]:
arr_ids = np.array([re.sub(r'\d+', '', d) for d in df_prof['username']])

username_feat_bow = get_vec(username_feat_bow_vectorizer, arr_ids)
username_feat_char = get_vec(username_feat_char_vectorizer, arr_ids)

print(username_feat_bow)
print(username_feat_char)

[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
